In [40]:
import web_scrapping
import db
import pandas as pd
from datetime import datetime

data = []
fecha_actual = datetime.now().strftime("%Y-%m-%d")

# Extraemos los productos y sus enlaces a la web de cada supermercado, y creamos un diccionario.     
consulta = "SELECT nombre_producto, link_coto, link_carrefour, link_dia FROM productos"
resultado = db.ejecutar_consulta(consulta)
productos = {}
for row in resultado:
    nombre_producto = row[0]
    enlaces = {
        "Coto": row[1],
        "Carrefour": row[2],
        "Día": row[3]
    }
    productos[nombre_producto] = enlaces

for producto, urls in productos.items():
    precios = {"producto": producto, "fecha": fecha_actual}
    for supermercado, url in urls.items():
        precio = None  # Inicializamos precio con el valor "null"
        try:
            if supermercado == "Coto":
                precio = web_scrapping.extraer_precio_coto(url)
            elif supermercado == "Carrefour":
                precio = web_scrapping.extraer_precio_carrefour(url)
            elif supermercado == "Día":
                precio = web_scrapping.extraer_precio_dia(url)
        except Exception as e:
            print(f"Error al extraer el precio del producto {producto} en {supermercado}: {e}")
        precios[supermercado] = precio if precio is not None else None
    data.append(precios)

df = pd.DataFrame(data)     # Convertimos a dataframe.
df.head()

Conexión cerrada correctamente.


,producto,fecha,Coto,Carrefour,Día
0,Azúcar Ledesma 1kg,2024-04-05,1399.0,1400.0,1395.0
1,Harina Morixe 0000 1kg,2024-04-05,870.0,Precio no encontrado,870.0
2,Harina Leudante Pureza 1 Kg,2024-04-05,1118.0,1120.0,1170.0
3,Arroz Integral Molinos Ala 500 Gr,2024-04-05,1519.0,1510.0,1510.0
4,Yerba Mate Playadito Suave 1 Kg,2024-04-05,4865.0,4105.0,3650.0


In [4]:
import db
import pandas as pd

try:
    consulta = '''SELECT DISTINCT ON (producto) producto, fecha, precio_coto, precio_carrefour, precio_dia
                    FROM precios_productos
                    ORDER BY producto, fecha DESC;'''
    precios = db.ejecutar_consulta(consulta)
    df_precios = pd.DataFrame(precios)
    df_precios = df_precios.drop(columns=df_precios.columns[0])  # Eliminar la primera columna, que es 'index'
    # Supongamos que quieres cambiar los nombres de las columnas 'precio_coto', 'precio_carrefour' y 'precio_dia' a minúsculas
    nuevos_nombres = {'precio_coto': 'Coto', 'precio_carrefour': 'Carrefour', 'precio_dia': 'Día'}
    # Utiliza el método rename() para cambiar los nombres de las columnas
    df_precios = df_precios.rename(columns=nuevos_nombres)
except Exception as e:
    print("Error al consultar los precios de los productos:", e)

df_precios

Conexión cerrada correctamente.


,fecha,Coto,Carrefour,Día
0,2024-04-06,2808.00,2805.00,2805.00
1,2024-04-06,1519.00,1350.00,1510.00
2,2024-04-06,1399.00,1400.00,1395.00
3,2024-04-06,809.00,609.00,709.00
4,2024-04-06,1118.00,1120.00,1170.00
5,2024-04-06,870.00,609.00,870.00
6,2024-04-06,809.00,None,709.00
7,2024-04-06,4865.00,4105.00,3650.00
